<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/small_convnet_data_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is for experimenting with data augmentation

# Notes

- Complete trial with eight layer model
- Add batch normalization
- Add more feedback to early stopping
  - accuracy and loss for that epoch
- Ideas for improving the performance

In [1]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor, Compose, Normalize
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import numpy as np
import random


In [2]:
aug_transforms = transforms.Compose([
  transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
  transforms.RandomRotation(degrees=45),
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomVerticalFlip(p=0.05),
  transforms.RandomGrayscale(p=0.2),
  transforms.ToTensor(),
  Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])
])

In [3]:
# Load the training data (CIFAR10 to start)
training_data = CIFAR100(
    root = "data",
    train = True,
    download = True,
    transform = aug_transforms
    #transform = Compose([ToTensor(),
    #                     Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])])
)

valid_data = CIFAR100(
    root = "data",
    train = True,
    download = True,
    transform = Compose([ToTensor(),
                         Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])])
)

# Load the test data
test_data = CIFAR100(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified
Files already downloaded and verified


In [4]:
training_targets = training_data.targets

In [5]:
train_split_index, valid_split_index = train_test_split(
    np.arange(len(training_targets)), test_size=0.2, stratify=training_targets
)

In [6]:
batch_size = 100

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size,
                              sampler=SubsetRandomSampler(train_split_index))
valid_dataloader = DataLoader(valid_data, batch_size=batch_size,
                              sampler=SubsetRandomSampler(valid_split_index))
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [7]:
# Check that it is splitting the data properly
train_length = 0
for _, y in train_dataloader:
  train_length += len(y)
print(f"Length of training split: {train_length}")

valid_length = 0
for _, y in valid_dataloader:
  valid_length += len(y)
print(f"Length of validation split: {valid_length}")

test_length = 0
for _, y in test_dataloader:
  test_length += len(y)
print(f"Length of test split: {test_length}")

Length of training split: 40000
Length of validation split: 10000
Length of test split: 10000


In [8]:
# Check that there are 100 instances of a random class in the validation set
count = 0
test_class = random.randint(0,99)
for _, y in valid_dataloader:
  for target in y:
    if int(target.numpy()) == test_class:
      count += 1

print(count)

100


# Building basic convolutional neural nets

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [10]:
# Define the model
class basicConvNet(nn.Module):
  def __init__(self, name):
    super(basicConvNet, self).__init__()
    self.name = name
    self.basic_conv_net = nn.Sequential(
        # 1st layer
        nn.Conv2d(3, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        nn.Dropout2d(p=0.1),
        
        # 2nd layer
        nn.Conv2d(50, 75, (3,3), padding='same'),
        nn.BatchNorm2d(75),
        nn.ReLU(),
        nn.Dropout2d(p=0.1),
        nn.MaxPool2d(2),
        
        # 3rd layer - 16x16
        nn.Conv2d(75, 100, (3,3), padding='same'),
        nn.BatchNorm2d(100),
        nn.ReLU(),
        nn.Dropout2d(p=0.1),
        
        # 4th layer
        nn.Conv2d(100, 150, (3,3), padding='same'),
        nn.BatchNorm2d(150),
        nn.ReLU(),
        nn.Dropout2d(p=0.1),
        nn.MaxPool2d(2),
        
        # 5th layer - 8x8
        nn.Conv2d(150, 200, (3,3), padding='same'),
        nn.BatchNorm2d(200),
        nn.ReLU(),
        nn.Dropout2d(p=0.1),
        
        # 6th layer
        nn.Conv2d(200, 250, (3,3), padding='same'),
        nn.BatchNorm2d(250),
        nn.ReLU(),
        nn.Dropout2d(p=0.1),
        nn.MaxPool2d(2),
        
        # 7th layer - 4x4
        nn.Flatten(),
        nn.Linear(250*4*4, 1000),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        
        # 8th layer
        nn.Linear(1000, 500),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(500, 100)
    )

  # Forward propagation
  def forward(self, x):
    x = self.basic_conv_net(x)
    #x = self.flatten(x)
    #x = self.output(x)
    return x
      

In [11]:
# Training loop
def train(dataloader, batch_size, model, loss_fn, optimizer):
  num_batches = len(dataloader)
  size = num_batches * batch_size
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [12]:
# Test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [13]:
# Validation function
def validation(dataloader, batch_size, model, loss_fn):
  num_batches = len(dataloader)
  size = num_batches * batch_size
  model.eval()
  val_loss, num_correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      val_loss += loss_fn(pred, y).item()
      num_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  val_loss /= num_batches
  accuracy = num_correct / size
  print(f"Validation Error: \n Validation accuracy: {(100 * accuracy):>0.1f}%, Validation loss: {val_loss:>8f} \n")
  return val_loss

In [14]:
n_epochs = 50

In [15]:
def training_loop(n_epochs, model, train_data, valid_data, batch_size,
                  loss_function, optimizer, early_stopping=False, patience=10):
  current_epoch = 0
  best_epoch = 0
  best_loss = float('inf')
  patience_counter = 0

  for e in range(n_epochs):
    print(f"\nEpoch {e+1}\n----------------------------")
    train(train_data, batch_size, model, loss_function, optimizer)
    val_loss = validation(valid_data, batch_size, model, loss_function)

    # Iterate epoch counter
    current_epoch += 1

    # If early_stopping check test_loss
    if early_stopping:
      # case: test loss beats the current best loss
      if val_loss < best_loss:
        # store loss
        best_loss = val_loss

        # reset patience counter
        patience_counter = 0

        # store model and epoch number
        print("Storing new best model.")
        best_model_state_dict = copy.deepcopy(model.state_dict)
        best_epoch = current_epoch

      # Case: patience limit not yet reached => iterate patience counter
      elif patience_counter < patience - 1:
        patience_counter += 1
        print(f"Patience count: {patience_counter}")

      # Case: patience limit reached
      else:
        print("Finished due to early stopping.")
        print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
        torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')
        break

  # If we get here, we did not stop early - save best model
  if early_stopping:
    print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
    torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')

In [16]:
model = basicConvNet('eight-layer').to(device)
model

basicConvNet(
  (basic_conv_net): Sequential(
    (0): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.1, inplace=False)
    (4): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout2d(p=0.1, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(75, 100, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (10): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Dropout2d(p=0.1, inplace=False)
    (13): Conv2d(100, 150, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (14): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): ReLU()
    (16): Dropout2d(p=0.1

In [17]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [18]:
training_loop(n_epochs, model, train_dataloader, valid_dataloader, batch_size,
              loss_fn, optimizer, early_stopping=True, patience=5)


Epoch 1
----------------------------
loss: 4.594431 [    0/40000]
loss: 4.617512 [10000/40000]
loss: 4.592182 [20000/40000]
loss: 4.445098 [30000/40000]
Validation Error: 
 Validation accuracy: 4.4%, Validation loss: 4.323768 

Storing new best model.

Epoch 2
----------------------------
loss: 4.289974 [    0/40000]
loss: 4.240777 [10000/40000]
loss: 4.202652 [20000/40000]
loss: 4.296968 [30000/40000]
Validation Error: 
 Validation accuracy: 7.8%, Validation loss: 4.059878 

Storing new best model.

Epoch 3
----------------------------
loss: 4.242836 [    0/40000]
loss: 4.097407 [10000/40000]
loss: 4.297120 [20000/40000]
loss: 4.021918 [30000/40000]
Validation Error: 
 Validation accuracy: 12.0%, Validation loss: 3.784455 

Storing new best model.

Epoch 4
----------------------------
loss: 4.097321 [    0/40000]
loss: 4.027771 [10000/40000]
loss: 3.986794 [20000/40000]
loss: 3.971786 [30000/40000]
Validation Error: 
 Validation accuracy: 14.7%, Validation loss: 3.619922 

Storing ne

# Results on CIFAR100:

## Eight layer model

### architecture
        # 1st layer
        nn.Conv2d(3, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        # 2nd layer
        nn.Conv2d(50, 75, (3,3), padding='same'),
        nn.BatchNorm2d(75),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        nn.MaxPool2d(2),
        # 3rd layer - 16x16
        nn.Conv2d(75, 100, (3,3), padding='same'),
        nn.BatchNorm2d(100),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        # 4th layer
        nn.Conv2d(100, 200, (3,3), padding='same'),
        nn.BatchNorm2d(200),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        nn.MaxPool2d(2),
        # 5th layer - 8x8
        nn.Conv2d(200, 100, (3,3), padding='same'),
        nn.BatchNorm2d(100),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        # 6th layer
        nn.Conv2d(100, 75, (3,3), padding='same'),
        nn.BatchNorm2d(75),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        nn.MaxPool2d(2),
        # 7th layer - 4x4
        nn.Conv2d(75, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
        # 8th layer
        nn.Conv2d(50, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        nn.Dropout2d(p=0.2),
    )
    self.flatten = nn.Flatten()
    self.output = nn.Linear(50*4*4, 100)

### Trial
- SGD lr = 5e-2
- Accuracy: 10.1%, Avg loss: 1.910425  @ epoch 56

---
### Architecture
        # 1st layer
        nn.Conv2d(3, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        # 2nd layer
        nn.Conv2d(50, 75, (3,3), padding='same'),
        nn.BatchNorm2d(75),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 3rd layer - 16x16
        nn.Conv2d(75, 100, (3,3), padding='same'),
        nn.BatchNorm2d(100),
        nn.ReLU(),
        # 4th layer
        nn.Conv2d(100, 200, (3,3), padding='same'),
        nn.BatchNorm2d(200),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 5th layer - 8x8
        nn.Conv2d(200, 100, (3,3), padding='same'),
        nn.BatchNorm2d(100),
        nn.ReLU(),
        # 6th layer
        nn.Conv2d(100, 75, (3,3), padding='same'),
        nn.BatchNorm2d(75),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 7th layer - 4x4
        nn.Conv2d(75, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
        # 8th layer
        nn.Conv2d(50, 50, (3,3), padding='same'),
        nn.BatchNorm2d(50),
        nn.ReLU(),
    )
    self.flatten = nn.Flatten()
    self.output = nn.Linear(50*4*4, 100)

### Trial
- SGD lr = 1e-1
- Accuracy: 9.3%, Avg loss: 2.115571 @ epoch 7


### Architecture
        # 1st layer
        nn.Conv2d(3, 256, (3,3), padding='same'),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        # 2nd layer
        nn.Conv2d(256, 128, (3,3), padding='same'),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 3rd layer - 16x16
        nn.Conv2d(128, 64, (3,3), padding='same'),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        # 4th layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 5th layer - 8x8
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        # 6th layer
        nn.Conv2d(32, 16, (3,3), padding='same'),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 7th layer - 4x4
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        # 8th layer
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.BatchNorm2d(16),
        nn.ReLU(),

### Results
- LR = 1e-1
- Accuracy: 8.2%, Avg loss: 2.274223 @ 11 epochs => decrease learning rate


### Architecture
        # 1st layer
        nn.Conv2d(3, 256, (3,3), padding='same'),
        nn.ReLU(),
        # 2nd layer
        nn.Conv2d(256, 128, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 3rd layer - 16x16
        nn.Conv2d(128, 64, (3,3), padding='same'),
        nn.ReLU(),
        # 4th layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 5th layer - 8x8
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.ReLU(),
        # 6th layer
        nn.Conv2d(32, 16, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 7th layer - 4x4
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.ReLU(),
        # 8th layer
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.ReLU(),

### Trials
- Accuracy: 6.6%, Avg loss: 2.713449 @ 27 epochs

## Six layer model

### Architecture
- 32 x 32 x 256 conv, ReLU - (3,3), 256
- Maxpool (2,2)
- 16 x 16 x 128 conv, ReLU - (3,3), 128
- Maxpool (2,2)
- 8 x 8 x 64 conv, ReLU - (3,3), 64
- Maxpool (2,2)
- 4 x 4 x 32 conv, ReLU - (3,3), 32
- 4 x 4 x 16 conv, ReLU - (3,3), 16
- Flatten
- Fully connect layer (256, 100)

### Trials
- 7.3% / 2.499734
  - SGD
  - `lr = 5e-2`
  - Validation accuracy - 7.3% @ 20 epochs
  - Avg loss - 2.499734
- 7.4% / 2.490613
  - SGD
  - `lr = 1e-2`
  - Validation accuracy - 7.4% @ 65 epochs
  - Avg loss - 2.490613


# Results on CIFAR10:

## Six layer model (current best)
- Test accuracy 72.7%
- Avg loss 1.733851

### Architecture
    # 1st layer
    nn.Conv2d(3, 256, (3,3), padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # 2nd layer
    nn.Conv2d(256, 128, (3,3), padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # 3rd layer
    nn.Conv2d(128, 64, (3,3), padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # 4th layer
    nn.Conv2d(64, 32, (3,3), padding='same'),
    nn.ReLU(),
    #nn.MaxPool2d((2,2)),
    # 5th layer
    nn.Conv2d(32, 32, (3,3), padding='same'),
    nn.ReLU(),
    #nn.MaxPool2d((2,2)),
    # 6th layer
    nn.Conv2d(32, 16, (3,3), padding='same'),
    nn.ReLU(),